# End-to-end Test of SOSS Simulations
This notebook will simulate SOSS data using `awesimsoss` and then quantify how well `specialsoss` can extract it.

## Imports

In [ ]:
# Imports
from astropy.modeling.models import BlackBody1D
from astropy.modeling.blackbody import FLAM
import astropy.units as q
from awesimsoss import TSO
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import numpy as np
from specialsoss import SossExposure
output_notebook()

## Simulation
First let's make a 1D blackbody spectrum.

In [ ]:
# Variables
teff = 2000*q.K

# Generate a blackbody at the given temperature from 0.5 - 3 um
bb = BlackBody1D(temperature=teff)
wave = np.linspace(0.5, 3., 1000)*q.um
flux = bb(wave).to(FLAM, q.spectral_density(wave))*1E-8

# Plot it
fig = figure(width=800, height=300, x_axis_label='Wavelength [um]', y_axis_label='Flux Density [{}]'.format(flux.unit))
fig.line(wave, flux, legend='Input Spectrum')
show(fig)

Lets make a SOSS simulation for this star with 2 integrations and 2 groups using `awesimsoss`.

In [ ]:
# Initialize the TSO object
sim = TSO(nints=2, ngrps=2, star=[wave, flux])

# Run the simulation
sim.simulate()

In [ ]:
# Run the plot method
sim.plot()

Now let's export the simulation to a pipeline ingestible '_uncal.fits' file.

In [ ]:
# Name the file
filename = 'SOSS_simulation_uncal.fits'

# Export the data
sim.export(filename)

## Reduction
Next let's load the "raw" data into `specialsoss` by passing a filename to the `SossExposure` class.

In [ ]:
# Initialize the exposure object with the '_uncal.fits' file
obs = SossExposure(filename)

We can calibrate the data using the JWST reduction pipeline with the `calibrate` method.

In [ ]:
# Run DETECTOR1 and SPEC2 pipelines
obs.calibrate()

In [ ]:
# Check out object info
obs.info

We can see the calibrated and uncalibrated data are stored as object properties (`uncal`, `rate`, `rateints`, `ramp`, `calints`, and `x1dints`) corresponding to the JWST pipeline dataproducts for SOSS mode, which can each be plotted and analyzed independently.

In [ ]:
# Inspect `rateints` data
# obs.rateints.data

## Extraction

Now let's perform the spectral extraction on the `rateints` data.

In [ ]:
# Run extraction method
obs.extract('sum', 'rateints', name='Extracted Spectrum')

We can take a look at the extracted spectra like so.

In [ ]:
# Plot extracted time series spectra
obs.plot_results('Extracted Spectrum')

Finally, let's compare the extracted spectrum for the first integration with the input spectrum.

In [ ]:
# Plot input spectrum...
fig = figure(width=800, height=300, x_axis_label='Wavelength [um]', y_axis_label='Flux Density [{}]'.format(flux.unit))
fig.line(wave, flux, legend='Input Spectrum')

# ...and extracted spectrum
wave_ext = obs.results['Extracted Spectrum']['wavelength']
flux_ext = obs.results['Extracted Spectrum']['flux'][0]
fig.line(wave_ext, flux_ext, legend='Extracted Spectrum')

show(fig)

Voila!